<a href="https://colab.research.google.com/github/RenataLReis/ChatBot/blob/main/Gerador_de_posts_para_o_LinkedIn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
!pip -q install google-genai
# Instala Framework de agentes do Google
!pip install -q google-adk

In [58]:
#Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [59]:
#Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [60]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import HTML, display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [50]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""

    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [61]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [62]:
##########################################
# --- Agente 1: Buscador de Artigos --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para trazer os artigos mais importantes sobre o tema fornecido.
        Foque em no máximo 10 artigos relevantes. O nível de relevância deve ser determinado pela quantidade de fontes citadas e
        pela relevância do publicador na área.
        Pelo menos 5 desses artigos devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [63]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um planejador de conteúdo, especialista em LinkedIn. Com base na lista de
        artigos mais recentes e relevantes do buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search])

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"

    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)

    return plano_do_post

In [64]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts profissionais para o LinkedIn.
            Você escreve posts para a engenheira de Software Renata Reis, especialista em .NET.
            Todos os posts devem ser escritos em língua inglesa e terem um tom profissional, mas que inspire curiosidade e engajamento.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para o LinkedIn sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem técnica e ser atrativo para recrutadores.
            Incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para LinkedIn")
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"

    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [65]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no LinkedIn.
            O público do post são outros desenvolvedores, mas também recrutadores e gerentes de tecnologia, então o tom deve ser profissional.
            Revise o rascunho de post de LinkedIn abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [66]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para LinkedIn com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para LinkedIn com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post: uso de mecanismos de busca feito pela IA
Maravilha! Vamos então criar o post sobre novidades em uso de mecanismos de busca feito pela IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para fornecer as informações mais relevantes sobre o uso de mecanismos de busca pela IA, farei algumas buscas no Google para identificar artigos recentes e relevantes sobre o tema.
> 
> 
> Com base nas pesquisas realizadas, aqui estão os principais pontos sobre o uso de mecanismos de busca pela IA:
> 
> **1. Impacto e Evolução da IA nos Mecanismos de Busca:**
> 
> *   **Personalização e Precisão:** A IA está revolucionando os mecanismos de busca, permitindo que eles entendam melhor a intenção do usuário e forneçam resultados mais personalizados e precisos. Algoritmos de IA como o RankBrain do Google e a IA do Bing interpretam o contexto e a semântica das perguntas, indo além das simples palavras-chave. [1, 2]
> *   **Processamento de Linguagem Natural (PLN) e Reconhecimento de Imagem:** Recursos como PLN e reconhecimento de imagem tornaram os mecanismos de busca mais inteligentes, melhorando a relevância dos resultados. [1]
> *   **Adaptação Contínua:** Os mecanismos de busca estão em constante evolução, com a IA desempenhando um papel crucial em seu desenvolvimento. [1]
> 
> **2. Exemplos de Aplicações da IA nos Mecanismos de Busca:**
> 
> *   **Google:** O Google se destaca como um dos principais exemplos de aplicação da IA, permitindo resultados de pesquisa mais rápidos e reconhecimento de sinônimos e assuntos relacionados. A IA aprende sobre as intenções de busca dos usuários, proporcionando uma melhoria contínua. [3]
> *   **Bing:** O Bing integra IA com sugestões alimentadas por IA e respostas inteligentes, oferecendo recomendações inteligentes e informações úteis. [1]
> *   **Chatbots com IA:** Chatbots com IA generativa estão se tornando mecanismos de resposta alternativos, substituindo consultas que antes eram feitas apenas por meio de mecanismos de pesquisa tradicionais. [5, 6]
> 
> **3. IA vs. Mecanismos de Busca Tradicionais:**
> 
> *   **Resultados Diretos vs. Listas de Links:** Mecanismos de pesquisa tradicionais exibem uma lista de links, enquanto os de IA geram resumos dos resultados, economizando tempo e esforço. [4]
> *   **Foco na Semântica vs. Palavras-Chave:** A pesquisa com IA se concentra na semântica, enquanto a pesquisa tradicional se concentra em palavras-chave. [4]
> *   **Experiências Personalizadas e Conversacionais:** A IA oferece experiências mais personalizadas e de conversação. [4]
> 
> **4. Desafios e Considerações:**
> 
> *   **Qualidade do Conteúdo:** Os algoritmos de mecanismos de pesquisa valorizam cada vez mais a qualidade do conteúdo para compensar a grande quantidade de informações geradas pela IA. [6]
> *   **Autenticidade do Conteúdo:** Há uma ênfase crescente na marca d'água e outros meios para autenticar o conteúdo de alto valor, com regulamentações governamentais responsabilizando as empresas pela identificação de materiais de marketing criados pela IA. [6]
> *   **Potencial de Viés:** Mecanismos de busca com IA dependem da qualidade dos dados nos quais foram treinados, e resultados podem ser tendenciosos ou imprecisos se os dados de treinamento forem falhos. [9]
> 
> **5. Ferramentas e Motores de Busca com IA:**
> 
> *   **Perplexity AI:** Oferece respostas concisas com fontes confiáveis, ideal para pesquisas acadêmicas e profissionais. [12, 20]
> *   **You.com:** Permite pesquisas baseadas em IA com diferentes modos e é altamente personalizável. [20]
> *   **Elicit:** Voltado para pesquisadores e estudantes, utiliza IA para revisar artigos científicos e encontrar referências. [20]
> *   **ChatGPT:** Evoluiu para se tornar uma ferramenta de pesquisa completa, com acesso à internet e habilidades de síntese. [20]
> *   **Google AI Mode:** Utiliza o modelo de linguagem Gemini 2.0 para fornecer resultados mais atualizados e precisos, com raciocínio e busca em tempo real. [15]
> *   **SearchGPT (OpenAI):** Combina IA generativa com informações atualizadas da web para uma experiência mais personalizada, mas está em fase de testes. [18]
> 
> **6. Tendências Futuras:**
> 
> *   **Adoção Crescente:** Espera-se que o uso de IA para pesquisa continue a crescer, com projeções de que 90 bilhões de adultos americanos usarão IA para pesquisa até 2027. [4, 8]
> *   **Marketing de Busca em Transformação:** O marketing de busca está perdendo espaço para a crescente adoção de chatbots de IA. [5]
> *   **IA como Protagonista:** A IA está se tornando a protagonista na experiência de busca, transformando a maneira como navegamos na internet. [5]
> 
> Em resumo, a IA está transformando os mecanismos de busca, tornando-os mais inteligentes, personalizados e eficientes. Embora existam desafios a serem superados, como a qualidade e autenticidade do conteúdo, a tendência é que a IA continue a desempenhar um papel cada vez mais importante na forma como encontramos informações online.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Com base nas suas pesquisas, aqui está um plano detalhado para um post no LinkedIn sobre o uso de mecanismos de busca pela IA, focando no tema mais relevante e abrangente:
> 
> **Tema escolhido:** Impacto e Evolução da IA nos Mecanismos de Busca
> 
> **Relevância:** Este tema é fundamental porque aborda a transformação contínua e o futuro da pesquisa online, algo que interessa a profissionais de diversas áreas.
> 
> **Tópicos a serem abordados no post:**
> 
> 1.  **Introdução:**
> 
>     *   **Título Atraente:** "Como a Inteligência Artificial Está Reinventando a Busca Online" ou "O Futuro da Busca: IA Personalizada e Eficiente".
>     *   **Contextualização:** Começar com uma pergunta provocadora sobre como as pessoas buscam informações atualmente e como isso está mudando.
>     *   **Tese:** Apresentar a IA como um divisor de águas nos mecanismos de busca, destacando sua capacidade de personalização e precisão.
> 2.  **O Impacto da IA na Personalização e Precisão:**
> 
>     *   **Explicação:** Detalhar como a IA permite que os mecanismos de busca compreendam melhor a intenção do usuário, indo além das palavras-chave.
>     *   **Exemplos:** Mencionar algoritmos como RankBrain do Google e a IA do Bing, explicando como eles interpretam o contexto e a semântica das perguntas.
>     *   **Dados:** Incluir estatísticas sobre o aumento da precisão nos resultados de busca devido à IA (se disponíveis).
> 3.  **Processamento de Linguagem Natural (PLN) e Reconhecimento de Imagem:**
> 
>     *   **PLN:** Explicar como o PLN permite que os mecanismos de busca entendam a linguagem humana, melhorando a relevância dos resultados.
>     *   **Reconhecimento de Imagem:** Mostrar como o reconhecimento de imagem possibilita buscar informações a partir de imagens, expandindo as possibilidades de pesquisa.
>     *   **Exemplos Práticos:** Apresentar exemplos de como esses recursos são utilizados no dia a dia.
> 4.  **Exemplos de Aplicações da IA em Mecanismos de Busca:**
> 
>     *   **Google:** Destacar como o Google utiliza a IA para oferecer resultados mais rápidos e precisos, reconhecendo sinônimos e assuntos relacionados.
>     *   **Bing:** Mencionar a integração da IA no Bing com sugestões e respostas inteligentes, oferecendo recomendações úteis.
>     *   **Chatbots com IA:** Explorar como os chatbots de IA generativa estão se tornando alternativas aos mecanismos de busca tradicionais, respondendo a perguntas diretamente.
> 5.  **IA vs. Mecanismos de Busca Tradicionais:**
> 
>     *   **Comparativo:** Apresentar as diferenças entre os mecanismos de busca tradicionais e os que utilizam IA.
>     *   **Resultados Diretos vs. Listas de Links:** Explicar como a IA gera resumos dos resultados, economizando tempo.
>     *   **Foco na Semântica vs. Palavras-Chave:** Detalhar como a IA se concentra na semântica, enquanto a pesquisa tradicional se concentra em palavras-chave.
>     *   **Experiências Personalizadas:** Mostrar como a IA oferece experiências mais personalizadas e conversacionais.
> 6.  **Desafios e Considerações:**
> 
>     *   **Qualidade do Conteúdo:** Enfatizar a importância da qualidade do conteúdo para compensar a grande quantidade de informações geradas pela IA.
>     *   **Autenticidade do Conteúdo:** Destacar a necessidade de autenticar o conteúdo de alto valor, mencionando a marca d'água e regulamentações governamentais.
>     *   **Potencial de Viés:** Alertar sobre o potencial de viés nos resultados de busca, dependendo da qualidade dos dados de treinamento da IA.
> 7.  **Ferramentas e Motores de Busca com IA:**
> 
>     *   **Perplexity AI:** Destacar suas respostas concisas com fontes confiáveis.
>     *   **You.com:** Mencionar sua personalização e diferentes modos de pesquisa.
>     *   **Elicit:** Apresentar sua utilidade para pesquisadores e estudantes.
>     *   **ChatGPT:** Explorar sua evolução como ferramenta de pesquisa completa.
>     *   **Google AI Mode:** Mencionar o uso do modelo Gemini 2.0 para resultados mais atualizados.
>     *   **SearchGPT (OpenAI):** Apresentar sua combinação de IA generativa com informações da web (em fase de testes).
> 8.  **Tendências Futuras:**
> 
>     *   **Adoção Crescente:** Projetar o aumento do uso de IA para pesquisa.
>     *   **Transformação do Marketing de Busca:** Discutir como o marketing de busca está se adaptando à crescente adoção de chatbots de IA.
>     *   **IA como Protagonista:** Enfatizar que a IA está se tornando central na experiência de busca.
> 9.  **Conclusão:**
> 
>     *   **Recapitulação:** Resumir os principais pontos abordados no post.
>     *   **Chamada para Ação:** Incentivar os leitores a experimentar os mecanismos de busca com IA e compartilhar suas experiências.
>     *   **Pergunta Final:** Abrir espaço para discussão, perguntando aos leitores sobre suas expectativas em relação ao futuro da busca com IA.
> 
> **Hashtags:**
> 
> *   #InteligenciaArtificial
> *   #MecanismosDeBusca
> *   #AI
> *   #BuscaOnline
> *   #Inovacao
> *   #Tecnologia
> *   #FuturoDaBusca
> *   #MarketingDigital
> 
> **Ferramentas de Busca Adicionais:**
> 
> Para enriquecer ainda mais o post, você pode pesquisar por:
> 
> *   **Estudos de Caso:** Exemplos de empresas que implementaram IA em seus mecanismos de busca e obtiveram sucesso.
> *   **Estatísticas Recentes:** Dados sobre o uso de IA em mecanismos de busca e seu impacto no comportamento do consumidor.
> 
> Com este plano detalhado, você estará pronto para criar um post no LinkedIn informativo e envolvente sobre o impacto da IA nos mecanismos de busca.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> Okay, I will create a draft LinkedIn post based on the provided plan. Here is the draft:
> 
> **Draft LinkedIn Post:**
> 
> Are you ready to redefine how you search for information? 
> 
> Artificial Intelligence (AI) is revolutionizing search engines, creating experiences that are not only more personalized but also significantly more efficient. Let's dive into how AI is reshaping the future of online search!
> 
> 🔍 **The AI Impact: Personalization and Precision**
> AI algorithms, such as Google’s RankBrain and Bing's AI, go beyond keywords to truly understand user intent. By interpreting context and semantics, AI delivers search results with unprecedented accuracy.
> 
> 🗣️ **NLP and Image Recognition**
> Natural Language Processing (NLP) enables search engines to understand human language, improving the relevance of search results. Image recognition expands search capabilities, allowing you to find information using images.
> 
> 🤖 **AI Applications in Search Engines**
> *   **Google:** Using AI to provide faster, more precise results by recognizing synonyms and related topics.
> *   **Bing:** Integrating AI to offer intelligent suggestions and answers, enhancing user experience.
> *   **AI Chatbots:** Generative AI chatbots are emerging as alternatives to traditional search engines, providing direct answers to your queries.
> 
> 🆚 **AI vs. Traditional Search Engines**
> AI-driven search engines offer direct result summaries, saving you time and focusing on semantics rather than just keywords. The result? A more personalized and conversational experience.
> 
> ⚠️ **Challenges and Considerations**
> While AI offers incredible benefits, it's crucial to address content quality, authenticity, and potential biases in search results. High-value content authentication and government regulations are vital.
> 
> 🛠️ **AI Search Tools to Explore**
> *   **Perplexity AI:** Known for its concise answers with reliable sources.
> *   **You.com:** Offers personalized search experiences with different modes.
> *   **Elicit:** A go-to for researchers and students.
> *   **ChatGPT:** Evolving into a comprehensive research tool.
> *   **Google AI Mode:** Using the Gemini 2.0 model for the freshest results.
> *   **SearchGPT (OpenAI):** Testing the combination of generative AI with web information.
> 
> 🚀 **Future Trends**
> Expect increased adoption of AI in search, transforming search engine marketing and placing AI at the heart of the search experience.
> 
> 💡 **What are your expectations for the future of AI-enhanced search?** Share your thoughts in the comments below! Let's discuss how AI is changing the way we find information.
> 
> #AI #SearchEngines #ArtificialIntelligence #FutureofSearch #Innovation #TechTrends
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!


--------------------------------------------------------------
